In [1]:
# pip install speechrecogniton
import speech_recognition as sr

In [2]:
r = sr.Recognizer()

# 1. 원본 음성 파일 STT

## 원하는 결과

미국과 중국의 대립은 역사에서 계속해서 반복 돼온 강대국들과의 싸움과는 다릅니다. 어느 쪽이 이길지 하는 이기는가 하는 싸움이 아닙니다. 어- 미국과 소련의 대립에 비교하면 미국과 중국은 어- 간단히 어- 나라가 분열된 게 분열된 상황이 아닙니다.

트럼프 대통령이 화웨이 중국의 화웨이 등 중국 제품을 미국에서 어- 전부 없애려고 해도 그건 미국 경제에도 악영향을 미치게 되고 중국도 피해를 입게 되는 상황이 벌어집니다.

In [15]:
sample = sr.AudioFile('C://Users/user/Desktop/sample.wav')

In [16]:
with sample as source:
    audio = r.record(source)
    
r.recognize_google(audio_data = audio, language = "ko-KR")

'오늘 밤 남극을 제외한 모든 대륙에서 코로나바이러스감염증 있으며 한국 한국에서는 현재 500원 이상해 새로운 가늠자가 추가 발생하고 있다 미국 검역소에서는 거 2만명의 사람들이 검사를 기다리고 있었다'

### Google STT 한계점
1. '어-' 등의 추임새를 표현하지 못함
2. 문단 구분 가능할까
3. '+'는 무엇..?

# 2. 침묵 구간 표시

In [7]:
from pydub import AudioSegment

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [8]:
from os import listdir
from os.path import isfile, join

In [9]:
from pydub.silence import split_on_silence

In [10]:
sound_file = AudioSegment.from_wav("C://Users/user/Desktop/sample.wav")
sound_file.dBFS

-27.41129033345151

In [11]:
# 침묵 구간 기준으로 자르기
audio_chunks = split_on_silence(
    # Use the loaded audio
    sound_file, 
    
    # must be silent for at least half a second
    min_silence_len=500,

    # consider it silent if quieter than -57 dBFS
    silence_thresh=-57
)

In [12]:
for i, chunk in enumerate(audio_chunks):
    out_file = ".//splitAudio//chunk{0}.wav".format(i)
    print("exporting", out_file)
    chunk.export(out_file, format="wav")

exporting .//splitAudio//chunk0.wav
exporting .//splitAudio//chunk1.wav
exporting .//splitAudio//chunk2.wav
exporting .//splitAudio//chunk3.wav
exporting .//splitAudio//chunk4.wav
exporting .//splitAudio//chunk5.wav
exporting .//splitAudio//chunk6.wav


In [14]:
for i in range(len(audio_chunks)):
    sample_chunk = sr.AudioFile('./splitAudio/chunk{0}.wav'.format(i))
    with sample_chunk as source:
        audio_chunk = r.record(source)
    result = r.recognize_google(audio_data = audio_chunk, language = "ko-KR")
    print(result,end=' [...] ')

미국과 중국의 대립은 역사에서 계속해서 반복되는 강대국들과의 싸움과는 다릅니다 [...] 어느 쪽이 일치하는 이기는가 하는 상황이 아니니 [...] 미국과 소련의 대립에 비교하면 미국과 중국은 [...] 간단히 [...] 나라가 분열되는 [...] 분열된 상황이 아님 [...] 트럼프 대통령이 화웨이 중국의 화웨이 등 중국 제품을 미국에서 전부 없애려고 해도 그건 미국 경제의 + 영향을 미치게 되고 중국도 피해를 입게 되는 상황이 벌어집니다 [...] 

### 말의 어미가 잘리는 경우가 발생함  
- 아닙니다 -> 아니니
- 아닙니다 -> 아님


In [67]:
type(audio_chunks[1])

pydub.audio_segment.AudioSegment

In [ ]:
# 침묵구간 길이 구하는 코드
def get_silence(audio, threshold, interval):
    "get length of silence in seconds from a wav file"

    # swap out pydub import for other types of audio
    song = AudioSegment.from_wav(audio)

    # break into chunks
    chunks = [song[i:i+interval] for i in range(0, len(song), interval)]

    # find number of chunks with dBFS below threshold
    silent_blocks = 0
    for c in chunks:
        if c.dBFS == float('-inf') or c.dBFS < threshold:
            silent_blocks += 1
        else:
            break

    # convert blocks into seconds
    return round(silent_blocks * (interval/1000), 3)

# get files in a directory
audio_path = 'path/to/directory'
audio_files = [i for i in listdir(audio_path) if isfile(join(audio_path, i))]

threshold = -80 # tweak based on signal-to-noise ratio

interval = 1 # ms, increase to speed up

leading_silences = {a: get_silence(join(audio_path, a),
                                   threshold, interval) for a in audio_files}

# to get tab-separated values:
for name, leading_silence in leading_silences.items():
    print(''.join([name, '\t', str(leading_silence)]))